## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-07-13-00-00 +0000,nypost,Inside the fentanyl ‘zombie zone’ public park ...,https://nypost.com/2025/12/07/us-news/how-las-...
1,2025-12-07-12-50-26 +0000,nypost,AI-powered police body cameras training to det...,https://nypost.com/2025/12/07/world-news/ai-po...
2,2025-12-07-12-47-16 +0000,nyt,Heathrow Airport Is Briefly Locked Down After ...,https://www.nytimes.com/2025/12/07/world/europ...
3,2025-12-07-12-34-45 +0000,nyt,Judge Stalls Justice Dept. Effort to Seek New ...,https://www.nytimes.com/2025/12/06/us/politics...
4,2025-12-07-12-33-07 +0000,nypost,Hochul will drive away New Yorkers if she supp...,https://nypost.com/2025/12/07/us-news/pataki-h...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2400/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
87,trump,24
120,fire,13
38,new,11
111,hong,7
112,kong,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
109,2025-12-06-20-49-19 +0000,nypost,Qatari leader warns Gaza cease-fire at ‘critic...,https://nypost.com/2025/12/06/world-news/qatar...,65
135,2025-12-06-18-10-02 +0000,nypost,Deadly Hong Kong fire came after residents wer...,https://nypost.com/2025/12/06/world-news/hong-...,52
11,2025-12-07-12-00-00 +0000,latimes,Why is FIFA President Gianni Infantino working...,https://www.latimes.com/sports/soccer/story/20...,47
127,2025-12-06-19-04-27 +0000,wapo,Trump urges a new vaccine schedule. Here’s wha...,https://www.washingtonpost.com/world/2025/12/0...,46
62,2025-12-07-05-41-09 +0000,nypost,Rosie O’Donnell’s therapist told her to ‘detac...,https://nypost.com/2025/12/07/media/rosie-odon...,44


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
109,65,2025-12-06-20-49-19 +0000,nypost,Qatari leader warns Gaza cease-fire at ‘critic...,https://nypost.com/2025/12/06/world-news/qatar...
135,39,2025-12-06-18-10-02 +0000,nypost,Deadly Hong Kong fire came after residents wer...,https://nypost.com/2025/12/06/world-news/hong-...
98,36,2025-12-06-23-09-00 +0000,wsj,Defense Secretary Pete Hegseth defended the U....,https://www.wsj.com/politics/national-security...
69,36,2025-12-07-03-46-56 +0000,nypost,Warner Bros. Discovery CEO’s bidding war destr...,https://nypost.com/2025/12/06/business/warner-...
4,30,2025-12-07-12-33-07 +0000,nypost,Hochul will drive away New Yorkers if she supp...,https://nypost.com/2025/12/07/us-news/pataki-h...
128,28,2025-12-06-18-58-09 +0000,bbc,Can you ban kids from social media? Australia ...,https://www.bbc.com/news/articles/crk7xgzj8y8o...
105,26,2025-12-06-21-41-29 +0000,nypost,Christmas returns to Bethlehem with tree-light...,https://nypost.com/2025/12/06/world-news/chris...
160,26,2025-12-06-14-35-09 +0000,nypost,Stephen Miller calls billion-dollar Minnesota ...,https://nypost.com/2025/12/06/us-news/stephen-...
82,23,2025-12-07-01-41-31 +0000,nypost,AR-15 allegedly found under Florida college st...,https://nypost.com/2025/12/06/us-news/ar-15-al...
47,23,2025-12-07-08-36-56 +0000,nypost,"At least 25 dead, including tourists, in fire ...",https://nypost.com/2025/12/07/world-news/india...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
